In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, csc_matrix

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Алгоритм LU-разложения
# https://math.wikia.org/ru/wiki/LU-%D1%80%D0%B0%D0%B7%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5
def LUdecomposition(A):
    n = A.shape[0]
    A_csr = csr_matrix(A)
    A_csc = A_csr.tocsc()
    L, U = csc_matrix(A_csr.shape), csr_matrix(A_csr.shape)
    for i in range(n):
        for j in range(i, n):
            U[0, i] = A_csr[0, i]
            L[i, 0] = A_csc[i, 0] / U[0, 0]
            t = 0
            for k in range(i):
                t += L[i, k] * U[k, j]
            U[i, j] = A_csr[i, j] - t
            if i <= j:
                t = 0
                for k in range(i):
                    t += L[j, k] * U[k, i]
                L[j, i] = (A_csc[j, i] - t) / U[i, i]
    return L, U

In [ ]:
# Метод Гаусса для верхнетреугольной матрицы A
def gaussian_method_u(A, b):
    n = A.shape[0]
    A = A.tocsr()
    b = b.copy()
    for i in range(n - 1, -1, -1):
        for j in range(i + 1, n):
            b[i] -= A[i, j] * b[j]
            A[i, j] = 0
        b[i] /= A[i, i]
        A[i, i] = 1
    return b


#  Метод Гаусса для нижнетреугольной матрицы A
def gaussian_method_d(A, b):
    n = A.shape[0]
    A = A.tocsr()
    b = b.copy()
    for i in range(n):
        for j in range(0, i):
            b[i] -= A[i, j] * b[j]
            A[i, j] = 0
        b[i] /= A[i, i]
        A[i, i] = 1
    return b

In [ ]:
# Алгоритм нахождения обратной матрицы
# https://www.cyberforum.ru/post12346418.html
def inv_matrix(A):
    n = A.shape[0]
    L, U = LUdecomposition(A)
    Y, A_inv = csc_matrix((n, n)), csc_matrix((n, n))
    E = csc_matrix(np.eye(n))
    for i in range(n):
        Y[:, i] = gaussian_method_d(L.copy(), E[:, i])
    for i in range(n):
        A_inv[:, i] = gaussian_method_u(U.copy(), Y[:, i])
    return A_inv

In [ ]:
# Алгоритм решения СЛАУ через LU-разложение
# https://ru.wikipedia.org/wiki/LU-%D1%80%D0%B0%D0%B7%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5
def solve_SOLA(A, b):
    A = A.copy()
    b = b.copy()
    L, U = LUdecomposition(A)
    Y = gaussian_method_d(L.copy(), b)
    X = gaussian_method_u(U.copy(), Y)
    return X

In [ ]:
# Итерационный метод Якоби для решения СЛАУ
# https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%AF%D0%BA%D0%BE%D0%B1%D0%B8
# http://bigor.bmstu.ru/?cnt/?doc=Parallel/ch030203.mod
def Yacobi_method(A, b, x=None, eps=1e-6, max_iter=1000):
    iter_cnt = 0
    n = A.shape[0]
    A = A.copy()
    b = b.copy()
    D, D_inv = csr_matrix((n, n)), csr_matrix((n, n))
    if x is None:
        x = np.zeros((n))
    for i in range(n):
        D[i, i] = A[i, i]
    B = D - A
    for i in range(n):
        D_inv[i, i] = 1 / A[i, i]
    f1, f2 = True, True
    for i in range(n):
        if abs(A[i, i]) * 2 <= np.sum(np.abs(A[i, :])):
            f1 = False
            print('Матрица не является матрицей с диагональным преобладанием')
            break
    if not (np.allclose(A.toarray(), A.toarray().T) and np.linalg.det((D + B).toarray())):
        f2 = False
        print('Не выполняется теорема 2')
    if f1 == False or f2 == False:
        print(f'Метод Якоби может не сойтись, лимит {max_iter} итераций')
    while max([abs(b[i] - A[i, :].dot(x)) for i in range(n)]) > eps and iter_cnt < max_iter:
        x = D_inv.dot(B.dot(x))+D_inv.dot(b)
        iter_cnt += 1
    print(f'На нахождение решения СЛАУ с точностью eps={eps} было затрачено {iter_cnt} итераций')
    return x

In [ ]:
# Размерность матрицы А
n = 5
# Максимальное по модулю число в сгенерированной матрице
mx = 10

In [ ]:
%%time
# Проверка на работу LU-разложения
A = csr_matrix(np.random.rand(n, n) * 2 * mx - mx)
L, U = LUdecomposition(A)
if n < 6:
    print('A:')
    for i in A.toarray():
        print(i)
    print('\nLU:')
    for i in L.dot(U).toarray():
        print(i)
    print('\n')
    for i in U.toarray():
        print(i)

else:
    print('Максимальная погрешность вычисления элемента матрицы: |e|=', np.max(np.abs(L.dot(U) - A)))
# По свойствам LU-разложения так же det(A) = det(LU) = det(L)*det(U) = det(U)
print('\ndef(A) =', np.linalg.det(A.toarray()))
print('def(LU) =', np.linalg.det(L.dot(U).toarray()))
print('def(L) * det(U) =', np.linalg.det(L.toarray()) * np.linalg.det(U.toarray()))
print('def(U) =', np.linalg.det(U.toarray()))

A:
[ 3.21211228 -3.70933933  6.70936719 -4.76601106 -4.32641501]
[-1.67660603 -3.03522067  0.85325303 -9.29350304 -2.2180771 ]
[ 2.32306217 -9.81249376 -0.0855647  -8.71197781  0.40017894]
[ 9.98893543 -2.31093189  2.61009036 -6.70855438 -8.80895889]
[-2.67729036  4.85625908  0.88114372  8.73551169 -7.07766015]

LU:
[ 3.21211228 -3.70933933  6.70936719 -4.76601106 -4.32641501]
[-1.67660603 -3.03522067  0.85325303 -9.29350304 -2.2180771 ]
[ 2.32306217 -9.81249376 -0.0855647  -8.71197781  0.40017894]
[ 9.98893543 -2.31093189  2.61009036 -6.70855438 -8.80895889]
[-2.67729036  4.85625908  0.88114372  8.73551169 -7.07766015]


[ 3.21211228 -3.70933933  6.70936719 -4.76601106 -4.32641501]
[  0.          -4.97136117   4.35529919 -11.78118784  -4.47630871]
[  0.           0.         -11.1841932   11.6312395    9.94895947]
[  0.           0.           0.         -24.32709371 -12.71024649]
[ 0.          0.          0.          0.         -9.80007049]

def(A) = 42578.476912846185
def(LU) = 42578.

In [ ]:
%%time
# Проверка на работу функции нахождения обратной матрицы
A = csr_matrix(np.random.rand(n, n) * 2 * mx - mx)
if n < 6:
    print('Встроенное нахождение обратной матрицы')
    for i in np.linalg.inv(A.toarray()):
        print(i)
    print('\nРеализованное нахождение обратной матрицы')
    for i in inv_matrix(A).toarray():
        print(i)
    print('\nПроверка, что действительно найдена обратная матрица (числа меньше 1е-12 приведены к нулю)')
    for i in inv_matrix(A).dot(A).toarray():
        print(np.array([x if x > 1e-12 else 0 for x in i]))
else:
    print('\nПроверка, что действительно найдена обратная матрица')
    print('Норма матрицы (A_inv * A - E) = ', np.max(np.abs(inv_matrix(A).dot(A) - np.eye(n))))

Встроенное нахождение обратной матрицы
[ 4.20167419 -0.94169328 -2.98859346  4.81436358 -6.36763447]
[ -9.67038233   2.12885066   7.05818457 -11.04140382  14.76237038]
[-7.46605486  1.61523006  5.39954378 -8.64734717 11.45817491]
[ -8.89390749   1.90197714   6.41738232 -10.15776583  13.62485233]
[-6.35984301  1.41233989  4.6114061  -7.34039688  9.8508429 ]

Реализованное нахождение обратной матрицы
[ 4.20167419 -0.94169328 -2.98859346  4.81436358 -6.36763447]
[ -9.67038233   2.12885066   7.05818457 -11.04140382  14.76237038]
[-7.46605486  1.61523006  5.39954378 -8.64734717 11.45817491]
[ -8.89390749   1.90197714   6.41738232 -10.15776583  13.62485233]
[-6.35984301  1.41233989  4.6114061  -7.34039688  9.8508429 ]

Проверка, что действительно найдена обратная матрица (числа меньше 1е-12 приведены к нулю)
[1. 0. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0.]
[0. 0. 0. 0. 1.]
CPU times: user 288 ms, sys: 109 ms, total: 397 ms
Wall time: 287 ms


In [ ]:
%%time
# Проверка на работу функции решения СЛАУ через LU-разложение
A = csr_matrix(np.random.rand(n, n) * 2 * mx - mx)
# A[n - 1, :] = np.zeros((n))   # Добавь эту строку, чтобы убедиться, что если ядро меньше размерности  - всё падает 
b = np.random.rand(n) * 2 * mx - mx
ans = solve_SOLA(A, b)
if n < 6:
    print('Встроенное решение СЛАУ:\tx = ', np.linalg.solve(A.toarray(), b))
    print('Реализованное решение СЛАУ:\tx = ', ans)
    print('b =\t', b)
    print('Ax =\t', A.dot(ans))
else:
    print('Норма вектора A*x-b = ', np.max(np.abs(A.dot(ans) - b)))

Норма вектора A*x-b =  1.7754686609805503e-12
CPU times: user 3.38 s, sys: 37.1 ms, total: 3.41 s
Wall time: 3.4 s


In [ ]:
%%time
# Проверка на работу функции итерационного решения СЛАУ - метод Якоби
A = csr_matrix(np.random.rand(n, n) * 2 * mx - mx)
for i in range(n):
    sm = 0
    for j in range(n):
        if i != j:
            sm += abs(A[i, j])
    A[i, i] = (1 if A[i, i] > 0 else -1) * (sm + 1)
# A - матрица с диагональным преобладанием(есть два стула)
# eps = 1e-6, начальное приближение - нули
# критерий остановки - норма(есть ещё евклидова норма)
b = np.random.rand(n) * 2 * mx - mx
ans = Yacobi_method(A, b)
if n < 6:
    print('Встроенное решение СЛАУ:\tx = ', np.linalg.solve(A.toarray(), b))
    print('Реализованное решение СЛАУ:\tx = ', ans)
    print('b =\t', b)
    print('Ax =\t', A.dot(ans))
else:
    print('Норма вектора A*x-b = ', np.max(np.abs(A.dot(ans) - b)))

Не выполняется теорема 2
Метод Якоби может не сойтись, лимит 1000 итераций
На нахождение решения СЛАУ с точностью eps=1e-06 было затрачено 10 итераций
Норма вектора A*x-b =  1.8322513151147746e-07
CPU times: user 195 ms, sys: 8.13 ms, total: 203 ms
Wall time: 188 ms


In [ ]:
n = 1000
k = 100

In [ ]:
%%time
# Проверка на работу функции итерационного решения СЛАУ - метод Якоби
A = csr_matrix(np.random.rand(n, n) * 2 * mx - mx)
for i in range(n):
    sm = 0
    for j in range(n):
        if i != j:
            sm += abs(A[i, j])
    A[i, i] = (1 if A[i, i] > 0 else -1) * (sm + 1)
# A - матрица с диагональным преобладанием(есть два стула)
# eps = 1e-6, начальное приближение - нули
# критерий остановки - норма(есть ещё евклидова норма)
b = A.dot(np.array([i + 1 for i in range(n)]))
ans = Yacobi_method(A, b, x=(np.random.rand(n) * 200 - 100),  max_iter=1000)
if n < 6:
    print('Встроенное решение СЛАУ:\tx = ', np.linalg.solve(A.toarray(), b))
    print('Реализованное решение СЛАУ:\tx = ', ans)
    print('b =\t', b)
    print('Ax =\t', A.dot(ans))
else:
    print('Норма вектора A*x-b = ', np.max(np.abs(A.dot(ans) - b)))

Не выполняется теорема 2
Метод Якоби может не сойтись, лимит 1000 итераций
На нахождение решения СЛАУ с точностью eps=1e-06 было затрачено 9 итераций
Норма вектора A*x-b =  4.824250936508179e-07
CPU times: user 24.7 s, sys: 106 ms, total: 24.8 s
Wall time: 24.8 s


In [ ]:
%%time
ans = solve_SOLA(A, b)
if n < 6:
    print('Встроенное решение СЛАУ:\tx = ', np.linalg.solve(A.toarray(), b))
    print('Реализованное решение СЛАУ:\tx = ', ans)
    print('b =\t', b)
    print('Ax =\t', A.dot(ans))
else:
    print('Норма вектора A*x-b = ', np.max(np.abs(A.dot(ans) - b)))